In [1]:
# IMPORT AND MOUNT DRIVE
import tensorflow as tf
import os
import os.path
import sys
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# LOAD TRAINING DATA
img_height = 128
img_width = 128
batch_size = 128 
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dir = '/content/drive/MyDrive/BIOINF/covid-project/dataset/train'

training_size = len(os.listdir(train_dir+"/covid-19")) +len(os.listdir(train_dir+"/normal")) + len(os.listdir(train_dir+"/viral-pneumonia"))
print("Training size ", training_size)

Train_Gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

train = Train_Gen.flow_from_directory(train_dir, 
                                      target_size = (img_height, img_width), 
                                      batch_size = batch_size, 
                                      class_mode = 'binary')

Training size  3443
Found 3443 images belonging to 3 classes.


In [3]:
#!rm -r "COVID-Chest-X-Rays-Deep-Learning-analysis"

In [4]:
# CLONE GITHUB REPOSITORY
!git clone https://github.com/Gialbo/COVID-Chest-X-Rays-Deep-Learning-analysis.git

Cloning into 'COVID-Chest-X-Rays-Deep-Learning-analysis'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 263 (delta 81), reused 178 (delta 28), pack-reused 0
Receiving objects: 100% (263/263), 28.68 MiB | 50.64 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [5]:
# LOAD DIRECTORY
import os
import os.path
import sys
sys.path.append('COVID-Chest-X-Rays-Deep-Learning-analysis/models')
!cd "COVID-Chest-X-Rays-Deep-Learning-analysis/models"

In [6]:
from rawGAN import rawGAN

In [7]:
model = rawGAN()

GAN model created


In [8]:
model.gan.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
model_1 (Functional)         (None, 128, 128, 3)       1807683   
_________________________________________________________________
model (Functional)           (None, 1)                 2618177   
Total params: 4,425,860
Trainable params: 1,774,531
Non-trainable params: 2,651,329
_________________________________________________________________


In [9]:
# noise used during training in order to evaluate how the network is learning
benchmarkNoise = model.generate_latent_points()

In [11]:
# set checkpoint directory
checkpoint_dir = '/content/drive/MyDrive/BIOINF/checkpoints_GAN/one-class'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(gan_optimizer=model.gan_optimizer,
                                 gan=model.gan)

In [ ]:
# train model
model.train_model(train, training_size, benchmarkNoise, checkpoint_prefix)

In [ ]:
# analyze losses and accuracies
model.plot_losses(model.history)
model.plot_losses(model.accuracy)